In [ ]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense,Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
import nltk
import re
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv('fake_news.csv')

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data.shape

(20800, 5)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,20800.0,10399.500000,6004.587135,0.0,5199.75,10399.5,15599.25,20799.0
label,20800.0,0.500625,0.500012,0.0,0.00,1.0,1.00,1.0


In [ ]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df=data.dropna()

In [ ]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
X=df.drop('label',axis=1)
y=df['label']

In [ ]:
sentences=X.copy()
sentences.reset_index(inplace=True)
ps = PorterStemmer()
corpus = []
for i in range(0,len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences['title'][i])#apart from words a-zA-Z , replace it with space #^=Not equal to
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [ ]:
vocab_size = 5000
onehot = [one_hot(words, vocab_size)for words in corpus]
onehot[:10]

[[3357, 706, 3155, 3329, 109, 2939, 310, 3877, 2919, 147],
 [2584, 2841, 1382, 1737, 1311, 2363, 2804],
 [1696, 1333, 2369, 497],
 [3391, 2610, 1807, 4460, 4094, 936],
 [2952, 1311, 2357, 4466, 1943, 4844, 1311, 1824, 3477, 3750],
 [1535,
  3038,
  3493,
  3283,
  2197,
  641,
  2930,
  3684,
  1679,
  3776,
  546,
  2587,
  4656,
  2387,
  2804],
 [3939, 3781, 3535, 129, 3321, 4698, 2084, 629, 1309, 1521, 3413],
 [3685, 3978, 1898, 1506, 557, 266, 641, 2723, 1309, 1521, 3413],
 [815, 1252, 4780, 1468, 4937, 4901, 4672, 2403, 641, 3266],
 [1948, 4812, 3215, 2134, 454, 4240, 3602, 4920]]

In [ ]:
length = 30
embedding=pad_sequences(onehot, maxlen=length, padding='pre')
embedding[:10]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 3357,  706,
        3155, 3329,  109, 2939,  310, 3877, 2919,  147],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 2584, 2841, 1382, 1737, 1311, 2363, 2804],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 1696, 1333, 2369,  497],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 3391, 2610, 1807, 4460, 4094,  936],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 2952, 1311,
        2357, 4466, 1943, 4844, 

In [ ]:
embedding_vector_features=40 
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features, input_length=length))
model.add(Bidirectional(LSTM(100)))
#model.add(GRU(100))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [ ]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X=np.array(embedding)
Y=np.array(y)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.33,random_state=1)
model.fit(X_train,y_train,epochs=10,batch_size=64,validation_data=(X_test,y_test) )

Epoch 1/10
192/192 [==============================] - 5s 17ms/step - loss: 0.0100 - accuracy: 0.9969 - val_loss: 0.4532 - val_accuracy: 0.9138
Epoch 2/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0039 - accuracy: 0.9991 - val_loss: 0.5609 - val_accuracy: 0.9140
Epoch 3/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0014 - accuracy: 0.9997 - val_loss: 0.5945 - val_accuracy: 0.9128
Epoch 4/10
192/192 [==============================] - 3s 13ms/step - loss: 4.9950e-04 - accuracy: 0.9999 - val_loss: 0.6207 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 2.6861e-04 - accuracy: 0.9999 - val_loss: 0.6518 - val_accuracy: 0.9132
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 1.3370e-04 - accuracy: 1.0000 - val_loss: 0.6758 - val_accuracy: 0.9123
Epoch 7/10
192/192 [==============================] - 3s 15ms/step - loss: 9.0054e-05 - accuracy: 1.0000 - val_loss: 0.6965 - val_

In [ ]:
y_pred=model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
y_pred

189/189 [==============================] - 1s 3ms/step


array([[1],
       [1],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
cm=confusion_matrix(y_test, y_pred) 
print(cm)


[[3165  276]
 [ 262 2332]]


In [ ]:
acc=print(accuracy_score(y_test, y_pred))
acc

0.9108533554266777
